In [19]:
# Import package for getting dataset example
import wooldridge as woo

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import scipy.stats as stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy as pt

import math
from tqdm import tqdm

# Introduction

> Recall that if homoscedasticity is violated, the standard errors are invalid and all inferences from t, F, and other test based on them are unreliable. Also the asymptotic of OLS depends on homoscedasticity.

This is the flow chart how to deal with heteroscedasticity:

1. Heteroscedasticity Tests.
2. If it's heteroscedasity, then try fit model with Weighted Least Squares (WLS) instead OLS.
3. Do Heteroscedasticity Test on WLS model.
4. If it's heteroscedasity, then try develop Heteroscedasticity-robust statistics.

# Heteroscedasticity-Robust Statistics

> Heteroscedasticity-Robust statistics will modify the standard error of the regression coefficients so that the hypothesis tests and confidence intervals remain valid even when heteroskedasticity is present. The modify standard error is called **Heteroscedasticity-robust standard error**.

NOTE: In case Heteroscedasticity, generally all inferences from t and F are not reliable. However wih some modification, there are t, F, or LM testing for case heteroscedasticity.

Here are a few key points about heteroskedasticity-robust statistics:

- Robust Standard Errors: These are adjusted standard errors that account for heteroskedasticity. They provide more reliable estimates of the variability of the regression coefficients when the error variance is not constant.

- Robust Variance-Covariance Matrix: The matrix of the parameter estimates' variances and covariances is adjusted to be robust to heteroskedasticity. This adjustment leads to more accurate inference in the presence of heteroskedasticity.

- White's Standard Errors: Named after Halbert White, these are a common type of heteroskedasticity-robust standard errors. They provide a way to correct the standard errors for heteroskedasticity without requiring a specific model for the error variance.

- Cluster-Robust Standard Errors: When data is grouped into clusters (e.g., students within schools, patients within hospitals), cluster-robust standard errors account for the fact that observations within the same cluster may be correlated. This is useful when heteroskedasticity is present and observations are not independent

In statsmodels.formula.api, if the regression model obtained by **ols** is stored in the variable $reg$, the variance-covariance
matrix can be calculated using:
- **reg.fit(cov_type='nonrobust')** or reg.fit() for the default homoscedasticity-based standard errors.
- **reg.fit(cov_type='HC0')** for the classical version of White’s robust variance-covariance matrix presented by Wooldridge (2019, Equation 8.4 in Section 8.2).
- **reg.fit(cov_type='HC1')** for a version of White’s robust variance-covariance matrix corrected by degrees of freedom.
- **reg.fit(cov_type='HC2')** for a version with a small sample correction. This is the default behavior of Stata.
- **reg.fit(cov_type='HC3')** for the refined version of White’s robust variance-covariance matrix.

![image](images/Example_8-2.png)

In [6]:
gpa3 = woo.dataWoo('gpa3')
# Part 1 demonstrate some commands for Heteroscedasticity-Robust Error

reg = smf.ols(formula='cumgpa ~ sat + hsperc + tothrs + female + black + white',
               data=gpa3, subset=(gpa3['spring'] == 1))

# Default modeling
model = reg.fit()

table_default = pd.DataFrame({'b': round(model.params, 4),
                              'se': round(model.bse, 4),
                              't': round(model.tvalues, 4), 
                              'pval': round(model.pvalues,)})

print("Table Default:")
print(table_default)

# Estimate model with White SE (only for spring data)
model_white = reg.fit(cov_type='HC0')
table_white = pd.DataFrame({'b': round(model_white.params, 4),
                              'se': round(model_white.bse, 4),
                              't': round(model_white.tvalues, 4), 
                              'pval': round(model_white.pvalues,)})

print("Table White:")
print(table_white)

# Estimate model with refined White SE (only for spring data)
model_refined = reg.fit(cov_type='HC3')

table_refined = pd.DataFrame({'b': round(model_refined.params, 4),
                              'se': round(model_refined.bse, 4),
                              't': round(model_refined.tvalues, 4), 
                              'pval': round(model_refined.pvalues,)})

print("Table Refined:")
print(table_refined)

Table Default:
                b      se       t  pval
Intercept  1.4701  0.2298  6.3971   0.0
sat        0.0011  0.0002  6.3885   0.0
hsperc    -0.0086  0.0012 -6.9060   0.0
tothrs     0.0025  0.0007  3.4255   0.0
female     0.3034  0.0590  5.1412   0.0
black     -0.1283  0.1474 -0.8705   0.0
white     -0.0587  0.1410 -0.4165   1.0
Table White:
                b      se       t  pval
Intercept  1.4701  0.2186  6.7261   0.0
sat        0.0011  0.0002  6.0136   0.0
hsperc    -0.0086  0.0014 -6.1001   0.0
tothrs     0.0025  0.0007  3.4136   0.0
female     0.3034  0.0586  5.1807   0.0
black     -0.1283  0.1181 -1.0863   0.0
white     -0.0587  0.1103 -0.5323   1.0
Table Refined:
                b      se       t  pval
Intercept  1.4701  0.2294  6.4089   0.0
sat        0.0011  0.0002  5.8402   0.0
hsperc    -0.0086  0.0014 -5.9341   0.0
tothrs     0.0025  0.0007  3.3418   0.0
female     0.3034  0.0600  5.0539   0.0
black     -0.1283  0.1282 -1.0007   0.0
white     -0.0587  0.1204 -0.4876   1

In [9]:
gpa = woo.dataWoo('gpa3')

# Part 2 demonstrate how to applying F-test on different Heteroscedasticity-Robust
#  and compare it.

reg = smf.ols(formula='cumgpa ~ sat + hsperc + tothrs + female + black + white',
               data=gpa3, subset=(gpa3['spring'] == 1))

hypotheses = ['black = 0', 'white = 0']

# F-tests using different variance-covariance formula
# 1. Usual VCOV
model = reg.fit()
ftest_default = model.f_test(hypotheses)
fstat_default = ftest_default.statistic
fpval_default = ftest_default.pvalue
print(f"fstat_default: {fstat_default}")
print(f"fpval_default: {fpval_default}")

# 2. Refined White VCOV
model_hc3 = reg.fit(cov_type='HC3')
ftest_hc3 = model_hc3.f_test(hypotheses)
fstat_hc3 = ftest_hc3.statistic
fpval_hc3 = ftest_hc3.pvalue
print(f"\nfstat_hc3: {fstat_hc3}")
print(f"fpval_hc3: {fpval_hc3}")

# 3. Classical White VCOV
model_hc0 = reg.fit(cov_type='HC0')
ftest_hc0 = model_hc0.f_test(hypotheses)
fstat_hc0 = ftest_hc0.statistic
fpval_hc0 = ftest_hc0.pvalue
print(f"\nfstat_hc3: {fstat_hc0}")
print(f"fpval_hc3: {fpval_hc0}")

fstat_default: 0.6796041956073365
fpval_default: 0.5074683622584049

fstat_hc3: 0.6724692957656628
fpval_hc3: 0.5110883633440992

fstat_hc3: 0.7477969818036214
fpval_hc3: 0.4741442714738484


# Heteroscedasticity Tests

The methods can be use:
1. Breusch-Pagan test for heteroscedasticity.


> The tests the hypothesis that the residual variance does not depend on the variables in x in the form **using simple linear relationship**.


2. White Test (more general)
> Checking heteroskedasticity **complex forms, such as quadratic relationships or interactions between independent variables**. Test hyptohesis that residual variance does not depend on any regressors in that complex form.

![image](images/Example_8-4.png)

In [18]:
# Calculation with manual

hprice1 = woo.dataWoo('hprice1')

# Estimate model
model = smf.ols(formula='price ~ lotsize + sqrft + bdrms',
                data=hprice1).fit()
table_results = pd.DataFrame({'b': round(model.params, 4),
                              'se': round(model.bse, 4),
                              't': round(model.tvalues, 4),
                              'pval': round(model.pvalues, 4)})
print("Table Result:")
print(table_results)

# Manual BP test (F version):
hprice1['resid_sq'] = model.resid ** 2
model_resid = smf.ols(formula='resid_sq ~ lotsize + sqrft + bdrms',
                      data=hprice1).fit()

## Calculate sum of squares
y_resid_mean = np.mean(hprice1['resid_sq'])
SSR = np.sum((model_resid.fittedvalues - y_resid_mean) ** 2)
SSE = np.sum(model_resid.resid ** 2)

## Degrees of freedom
df_regression = model_resid.df_model
df_residual = model_resid.df_resid

## Mean Squares
MSR = SSR / df_regression
MSE = SSE / df_residual

## F-statistic
bp_F_statistic = MSR / MSE

## p-value
bp_F_pval = 1 - stats.f.cdf(F_statistic, df_regression, df_residual)

print(f'F_statistic: {bp_F_statistic}')
print(f'p_value: {bp_F_pval}')


# Manual BP test (LM version):
# hprice1['resid_sq'] = model.resid ** 2
model_resid = smf.ols(formula='resid_sq ~ lotsize + sqrft + bdrms',
                      data=hprice1).fit()
n = len(hprice1)
r2_aux = model_resid.rsquared
bp_LM_statistic = n * r2_aux
bp_LM_pval = 1 - stats.chi2.cdf(bp_LM_statistic, 
                                df=len(model_resid.params) - 1)

print(f"bp_LM_statistic: {bp_LM_statistic}")
print(f"bp_LM_pval: {bp_LM_pval}")

Table Result:
                 b       se       t    pval
Intercept -21.7703  29.4750 -0.7386  0.4622
lotsize     0.0021   0.0006  3.2201  0.0018
sqrft       0.1228   0.0132  9.2751  0.0000
bdrms      13.8525   9.0101  1.5374  0.1279
F_statistic: 5.338919363241526
p_value: 0.0020477444209358042
bp_LM_statistic: 14.092385504350437
bp_LM_pval: 0.002782059555688887


In [25]:
# Calculation with Automatic

hprice1 = woo.dataWoo('hprice1')

# Estiimate model
model = smf.ols(formula='price ~ lotsize + sqrft + bdrms',
                data=hprice1).fit()
table_results = pd.DataFrame({'b': round(model.params, 4),
                              'se': round(model.bse, 4),
                              't': round(model.tvalues, 4),
                              'pval': round(model.pvalues, 4)})
print("Table Result:")
print(table_results)


# Automatic BP test (LM Version and F version)
y, X_bp = pt.dmatrices('price ~ lotsize + sqrft + bdrms',
                       data=hprice1, return_type='dataframe')
results_bp = sm.stats.diagnostic.het_breuschpagan(model.resid, X_bp)
bp_LM_statistic, bp_LM_pval, bp_F_statistic, bp_F_pval = results_bp


print(f'F_statistic: {bp_F_statistic}')
print(f'p_value: {bp_F_pval}')

print(f"bp_LM_statistic: {bp_LM_statistic}")
print(f"bp_LM_pval: {bp_LM_pval}")

Table Result:
                 b       se       t    pval
Intercept -21.7703  29.4750 -0.7386  0.4622
lotsize     0.0021   0.0006  3.2201  0.0018
sqrft       0.1228   0.0132  9.2751  0.0000
bdrms      13.8525   9.0101  1.5374  0.1279
F_statistic: 5.338919363241507
p_value: 0.002047744420935856
bp_LM_statistic: 14.092385504350437
bp_LM_pval: 0.0027820595556888317


![image](images/Example_8-5.png)

In [30]:
hprice1 = woo.dataWoo('hprice1')

# Estimate model
model = smf.ols(formula='np.log(price) ~ np.log(lotsize) + np.log(sqrft) + bdrms',
                data=hprice1).fit()

# BP test
y, X_bp = pt.dmatrices('np.log(price) ~ np.log(lotsize) + np.log(sqrft) + bdrms',
                       data=hprice1, return_type='dataframe')
result_bp = sm.stats.diagnostic.het_breuschpagan(model.resid, X_bp)
bp_LM_statistic = result_bp[0]
bp_LM_pval = result_bp[1]

print(f"bp_LM_statistic: {bp_LM_statistic}")
print(f"bp_LM_pval: {bp_LM_pval}")

# White test
X_wh = pd.DataFrame({'const': 1, 'fitted_reg': model.fittedvalues,
                     'fitted_reg_sq': model.fittedvalues ** 2})
result_white = sm.stats.diagnostic.het_breuschpagan(model.resid, X_wh)
white_statistic = result_white[0]
white_pval = result_white[1]

print(f"white_statistic: {white_statistic}")
print(f"white_pval: {white_pval}")

bp_LM_statistic: 4.223245741805276
bp_LM_pval: 0.23834482631492962
white_statistic: 3.4472865468751914
white_pval: 0.17841494794131688


# Wighted Least Squares

> It's process for fitting model but instead minimize Sum of Squared Residuals (OLS), we use Weighted Sum.

WLS is a special case of Feasible Generalized Least Squares (FGLS):

- FGLS is a more general approach that adjusts the OLS estimates to account for heteroskedasticity and other forms of error structure.

- In the context of heteroskedasticity, FGLS estimates the variance of the errors first and then applies WLS using these estimated variances as weights. 

- This process makes the estimation "feasible" because the true variances of the errors are usually unknown and must be estimated from the data.

![image](images/Example_8-6.png)

In [37]:
k401ksubs = woo.dataWoo('401ksubs')

# Subsetting data
k401ksubs_sub = k401ksubs[k401ksubs['fsize'] == 1]

# OLS (only for singles, i.e. fsize==1)
model_ols = smf.ols(formula='nettfa ~ inc + I((age - 25) ** 2) + male + e401k',
                    data=k401ksubs_sub).fit()

table_ols = pd.DataFrame({'b': round(model_ols.params, 4),
                          'se': round(model_ols.bse, 4), 
                          't': round(model_ols.tvalues, 4),
                          'pval': round(model_ols.pvalues, 4)})
print(table_ols)

# WLS
wls_weight = list(1 / k401ksubs_sub['inc'])
reg_wls = smf.wls(formula='nettfa ~ inc + I((age-25)**2) + male + e401k',
                  weights=wls_weight, data=k401ksubs_sub)
model_wls = reg_wls.fit()
table_wls = pd.DataFrame({'b': round(model_wls.params, 4),
                          'se': round(model_wls.bse, 4), 
                          't': round(model_wls.tvalues, 4),
                          'pval': round(model_wls.pvalues, 4)})
print(table_wls)

                          b      se        t    pval
Intercept          -20.9850  2.4720  -8.4890  0.0000
inc                  0.7706  0.0615  12.5396  0.0000
I((age - 25) ** 2)   0.0251  0.0026   9.6888  0.0000
male                 2.4779  2.0478   1.2101  0.2264
e401k                6.8862  2.1233   3.2432  0.0012
                          b      se        t    pval
Intercept          -16.7025  1.9580  -8.5304  0.0000
inc                  0.7404  0.0643  11.5140  0.0000
I((age - 25) ** 2)   0.0175  0.0019   9.0796  0.0000
male                 1.8405  1.5636   1.1771  0.2393
e401k                5.1883  1.7034   3.0458  0.0024


**WLS Robust**

In [43]:
k401ksubs = woo.dataWoo('401ksubs')

# Subsetting data
k401ksubs_sub = k401ksubs[k401ksubs['fsize'] == 1]

# WLS
wls_weight = list(1 / k401ksubs_sub['inc'])
reg_wls = smf.wls(formula='nettfa ~ inc + I((age-25)**2) + male + e401k',
                  weights=wls_weight, data=k401ksubs_sub)
model_wls = reg_wls.fit()
table_wls = pd.DataFrame({'b': round(model_wls.params, 4),
                          'se': round(model_wls.bse, 4), 
                          't': round(model_wls.tvalues, 4),
                          'pval': round(model_wls.pvalues, 4)})
print(table_wls)

# Robust results (Refined White SE)
model_white = reg_wls.fit(cov_type='HC3')
table_white = pd.DataFrame({'b': round(model_white.params, 4),
                          'se': round(model_white.bse, 4), 
                          't': round(model_white.tvalues, 4),
                          'pval': round(model_white.pvalues, 4)})
print(table_white)

                          b      se        t    pval
Intercept          -16.7025  1.9580  -8.5304  0.0000
inc                  0.7404  0.0643  11.5140  0.0000
I((age - 25) ** 2)   0.0175  0.0019   9.0796  0.0000
male                 1.8405  1.5636   1.1771  0.2393
e401k                5.1883  1.7034   3.0458  0.0024
                          b      se       t    pval
Intercept          -16.7025  2.2482 -7.4292  0.0000
inc                  0.7404  0.0752  9.8403  0.0000
I((age - 25) ** 2)   0.0175  0.0026  6.7650  0.0000
male                 1.8405  1.3132  1.4015  0.1611
e401k                5.1883  1.5743  3.2955  0.0010


![image](images/Example_8-7.png)

In [44]:
smoke = woo.dataWoo('smoke')

# OLS
model_ols = smf.ols(formula='cigs ~ np.log(income) + np.log(cigpric) +' 
                            'educ + age + I(age**2) + restaurn',
                    data=smoke).fit()
table_ols = pd.DataFrame({'b': round(model_ols.params, 4),
                          'se': round(model_ols.bse, 4),
                          't': round(model_ols.tvalues, 4),
                          'pval': round(model_ols.pvalues, 4)})

print(f"Table OLS:\n", table_ols)

# BP test
y, X = pt.dmatrices('cigs ~ np.log(income) + np.log(cigpric) + educ +'
                    'age + I(age**2) + restaurn',
                    data=smoke, return_type='dataframe')
result_bp = sm.stats.diagnostic.het_breuschpagan(model_ols.resid, X)
bp_statistic = result_bp[0]
bp_pval = result_bp[1]
print(f"bp_statistic: {bp_statistic}")
print(f"bp_pval: {bp_pval}")

# FGLS (esimation of the variance function)
smoke['logu2'] = np.log(model_ols.resid ** 2)
model_fgls = smf.ols(formula='logu2 ~ np.log(income) + np.log(cigpric) +'
                             'educ + age + I(age**2) + restaurn',
                     data=smoke).fit()
table_fgls = pd.DataFrame({'b': round(model_ols.params, 4),
                          'se': round(model_ols.bse, 4),
                          't': round(model_ols.tvalues, 4),
                          'pval': round(model_ols.pvalues, 4)})

print(f"Table FGLS:\n", table_fgls)

# FGLS fitted values as weight in WLS
wls_weight = list(1 / np.exp(model_fgls.fittedvalues))
model_wls = smf.wls(formula='cigs ~ np.log(income) + np.log(cigpric) +'
                            'educ + age + I(age**2) +restaurn',
                     weights=wls_weight, data=smoke).fit()
table_wls = pd.DataFrame({'b': round(model_wls.params, 4),
                          'se': round(model_wls.bse, 4),
                          't': round(model_wls.tvalues, 4),
                          'pval': round(model_wls.pvalues, 4)})

print(f"Table WLS:\n", table_wls)

Table OLS:
                       b       se       t    pval
Intercept       -3.6398  24.0787 -0.1512  0.8799
np.log(income)   0.8803   0.7278  1.2095  0.2268
np.log(cigpric) -0.7509   5.7733 -0.1301  0.8966
educ            -0.5015   0.1671 -3.0016  0.0028
age              0.7707   0.1601  4.8132  0.0000
I(age ** 2)     -0.0090   0.0017 -5.1765  0.0000
restaurn        -2.8251   1.1118 -2.5410  0.0112
bp_statistic: 32.25841908120112
bp_pval: 1.4557794830279539e-05
Table FGLS:
                       b       se       t    pval
Intercept       -3.6398  24.0787 -0.1512  0.8799
np.log(income)   0.8803   0.7278  1.2095  0.2268
np.log(cigpric) -0.7509   5.7733 -0.1301  0.8966
educ            -0.5015   0.1671 -3.0016  0.0028
age              0.7707   0.1601  4.8132  0.0000
I(age ** 2)     -0.0090   0.0017 -5.1765  0.0000
restaurn        -2.8251   1.1118 -2.5410  0.0112
Table WLS:
                       b       se       t    pval
Intercept        5.6355  17.8031  0.3165  0.7517
np.log(income)   